# Extract Data Sources
Parse the two data sources [QM9](https://www.nature.com/articles/sdata201422) and QM9-G4MP2 into a more-usable format.

In [1]:
from jcesr_ml.utils import compute_atomization_energy
from dlhub_sdk.models.datasets import TabularDataset
from ase.io.xyz import write_xyz
from ase.db import connect
from tarfile import TarFile
from math import isclose
from io import StringIO
from tqdm import tqdm
import pandas as pd
import numpy as np
import json
import glob
import sys
import bz2
import re
import os

Things to change

In [2]:
g4mp2_path = os.path.join('data', 'input', 'g4mp2-gdb9.db') 

In [3]:
qm9_path = os.path.join('data', 'input', 'dsgdb9nsd.xyz.tar.bz2')

In [4]:
output_path = os.path.join('data', 'output', 'g4mp2_data.json.gz')

In [5]:
evil_mol_list = os.path.join('data', 'input', 'uncharacterized.txt')  # Molecules that changed during relaxation

In [6]:
unrelaxed_mols = os.path.join('data', 'input', 'negative_freqs.txt')  # Molecules that are incompletely relaxed

## Show an Example Entry from Each Dataset
Just print a single row to show what we're dealing with

### QM9
QM9 is stored in a tar file, we will print a single file. As described in the [QM9 Paper](https://www.nature.com/articles/sdata201422), the data for each atom is encoded in the 2nd line of the file

In [7]:
qm9_tar = TarFile.open(fileobj=bz2.open(qm9_path))

In [8]:
%%time
print(qm9_tar.extractfile('dsgdb9nsd_002348.xyz').read().decode())

17
gdb 2348	5.28659	1.8222	1.57619	2.635	64.78	-0.2389	-0.0093	0.2295	836.4733	0.147702	-309.732528	-309.724906	-309.723961	-309.765172	27.004	
C	-0.2803079247	 1.5743791367	-0.1679267063	-0.480689
C	 0.0931773117	 0.128156842	 0.1059703556	 0.370419
O	 1.2326500558	-0.2035601694	 0.3477764467	-0.303524
C	-1.031822354	-0.8828815436	 0.0167435484	-0.062595
C	-1.256139412	-1.5157994179	-1.4043788362	-0.223367
C	-1.4413992056	-2.8933937147	-0.7149243705	-0.208499
C	-0.7792120195	-2.3105928746	 0.5613611976	-0.229108
H	 0.6173178997	 2.1900427681	-0.2366790156	 0.140066
H	-0.8646816861	 1.6561455321	-1.0912545828	 0.136658
H	-0.9169523951	 1.949015146	 0.6426317143	 0.140358
H	-1.9654849722	-0.4472240448	 0.3863693781	 0.073406
H	-2.0828364449	-1.1232768693	-2.0005614542	 0.101245
H	-0.3397046457	-1.4779519226	-1.9999905269	 0.116103
H	-2.4983631602	-3.1272363355	-0.5592465532	 0.10216
H	-0.9620686148	-3.7549824633	-1.1850114617	 0.10202
H	 0.2951071718	-2.5037052497	 0.6020920261	 0.12806

Random access from the tar is slow, so let's cache the data

In [9]:
qm9_xyz = dict((e.name, qm9_tar.extractfile(e).read().decode()) for e in tqdm(qm9_tar.getmembers()))

100%|██████████| 133885/133885 [00:17<00:00, 7821.91it/s]


## Get the QM9-G4MP2 Data
The G4MP2 data is encoded in a ASE DB. 

In [10]:
db = connect(g4mp2_path)

In [11]:
row = db.get(1)
for key in row:
    print('{:22s}: {}'.format(key, row[key]))

g4mp2_Standard_Enthalpy_Formation: 35.19166
g4mp2_Enthalpy        : -437.311525
Smiles                : O[C@H]1[C@H]2[C@H]3O[C@H]4[C@@H]1N2[C@@H]34
Plain_InChI           : InChI=1S/C6H7NO2/c8-4-1-5-3-6(9-5)2(4)7(1)3/h1-6,8H
g4mp2_ZPE             : 0.125447
g4mp2_FreeE           : -437.348073
g4mp2_AtomizationE    : 1569.140848
Plain_Smiles          : OC1C2C3OC4C1N2C34
g4mp2_E0K             : -437.318254
g4mp2_Energy          : -437.312469
InChI                 : InChI=1S/C6H7NO2/c8-4-1-5-3-6(9-5)2(4)7(1)3/h1-6,8H/t1-,2+,3-,4-,5+,6-
gdbID                 : 79782
id                    : 1
unique_id             : 65ea76a1cdd7b1ec5bbeef5989edbc5e
ctime                 : 19.17499604994695
mtime                 : 19.17499604994695
user                  : bnarayanan
numbers               : [8 6 6 6 8 6 6 7 6 1 1 1 1 1 1 1]
positions             : [[ 0.10321337  1.43815203 -0.09291334]
 [-0.01441036  0.05632053  0.02841368]
 [ 0.21133207 -0.67331038  1.37509802]
 [-0.75619358 -0.13873025  2.48

There is a lot of information packed in here, and we need to extract it into a usable form

## Create Utility Functions
We need a series of functions to extract data from the database. Besides the XYZ coordinates, we also have various properties of the molecule and its InChI. 

The properties are in a special order, defined by the person who ran these calculations. By consulting with them, we created extractors for pulling this information out of the files

In [12]:
def extract_qm9_data(lines):
    """Extract QM9 data from an XYZ file
    
    Args:
        lines ([string]): Lines from the XYZ file
    Returns:
        (dict): Key properties from the xyz file
    """
    
    # Split the data line
    data_line = lines[1].split()
    
    # Store the index and number of atoms
    output = {'n_atom': int(lines[0].split()[0]),
              'index': int(data_line[1])}
    
    # Store all of the other properties
    properties = ["A", "B", "C", "mu", "alpha", "homo", "lumo", 
                  "bandgap", "R2", "zpe", "u0", "u", "h", "g", "cv"]
    for name, value in zip(properties, data_line[2:]):
        output[name] = float(value)
    
    return output

In [13]:
def extract_charges(lines):
    """Extract the charges on each of the atoms
    
    Args:
        lines ([string]): Lines from the XYZ file
    Returns:
        ([float]) Charges on each atom
    """
    
    n_atoms = int(lines[0])
    return list(map(float, [x.split()[-1].replace("*^", "e") for x in lines[2:(2+n_atoms)]]))

In [14]:
def extract_g4mp2(mol):
    """Extract the G4MP2 data from the DB row, store in standard names
    
    Args:
        row: Row from the ASE db
    Returns:
        (dict): Properites related to the G4MP2 otuputs
    """
    properties = {
        'g4mp2_Standard_Enthalpy_Formation': 'g4mp2_hf298',
        'g4mp2_Enthalpy': 'g4mp2_enthalpy',
        'g4mp2_ZPE' : 'g4mp2_zpe',
        'g4mp2_FreeE': 'g4mp2_free',
        'g4mp2_E0K': 'g4mp2_0k',
        'g4mp2_Energy': 'g4mp2_energy'
    }
    row = mol.info['key_value_pairs']
    return dict((name, row[key]) for key, name in properties.items())

In [15]:
def extract_structure(lines):
    """Extract the smiles and InChi strings
    
    Args:
        lines ([string]): Appropriate lines from the QM9 XYZ file
    Returns:
        (dict): Smiles and INCHi strings
    """
    
    smiles = lines[0].split('\t')
    inchi = lines[1].split('\t')
    return {
        'smiles_0':smiles[0],
        'smiles_1':smiles[1],
        'inchi_0':inchi[0],
        'inchi_1':inchi[1]
    }

In [16]:
def get_clean_xyz_file(mol):
    """Get the atoms
    
    Args:
        mol (ase.Atoms): Molecule to evaluate
    Returns:
        (string) File in a cleaner XYZ format
    """
    strio = StringIO()
    write_xyz(strio, mol)
    return strio.getvalue()

In [17]:
def get_counts(mol):
    """Get th size of teh molecule
    
    Args:
        mol (ase.Atoms): Molecule to evaluate
    Returns:
        (dict) Different measures of molecular size:
            n_heavy_atom (int): Number of heavy (i.e., non-Hydrogen) atoms
            n_electrons (int): Number of electrons in the system
    """
    Z = mol.get_atomic_numbers()
    return {
        'n_heavy_atoms': (Z > 1).sum(),
        'n_electrons': Z.sum(), 
    }

In [18]:
def get_atomization_energies(mol, u0, g4mp2_0k):
    """Compute the atomization energies for each molecule
    
    Args:
        mol (ase.Atoms): Molecule to evaluate
        u0 (float): B3LYP 0K total energy
        g4mp2_0k (float): G4MP2 total energy
    Returns:
        (dict) With computed total energies:
            u0_atom (float): B3LYP atomization energy (Ha)
            g4mp2_atom (float): B3LYP atomization energy (Ha)
    """
    return {
        'u0_atom': compute_atomization_energy(mol, u0, 'b3lyp'),
        'g4mp2_atom': compute_atomization_energy(mol, g4mp2_0k, 'g4mp2')
    }

In [19]:
def parse_data(db, qm9_xyz):
    """Generate dataset as a dataframe
    
    Args:
        db: ASE DB with G4MP2 data
        qm9_xyz (dict): All of the files in the QM9 tar archive
    Returns:
        (DataFrame) Dataset
    """
    
    matrix = []
    for row in tqdm(db.select(), total=len(db)):
        # Make the atoms object out of the row
        mol = row.toatoms(add_additional_information=True)
        
        # Get the matching QM9 XYZ file
        file = 'dsgdb9nsd_{:06d}.xyz'.format(mol.info['key_value_pairs']['gdbID'])
        lines = qm9_xyz[file].split("\n")

        # Get the appropriate lines from the file
        xyz_data = lines
        structure_data = lines[-3:]

        # Extract the numerical data
        item = extract_qm9_data(xyz_data)
        item.update(extract_g4mp2(mol))
        item['atomic_charges'] = extract_charges(xyz_data)

        # Load the structure as InCHi and SMILES
        item.update(extract_structure(structure_data))

        # Store the file name and XYZ file
        item['filename'] = os.path.basename(file)
        item['xyz'] = get_clean_xyz_file(mol)

        # Get some metrics of the atomic size
        item.update(get_counts(mol))

        # Get the atomization energies
        item.update(get_atomization_energies(mol,
                                             item['u0'],
                                             item['g4mp2_0k']))

        matrix.append(item)

    df = pd.DataFrame(matrix)
    
    return df     

## Parse the Entire Dataset
Run the extraction on all the files

In [20]:
df = parse_data(db, qm9_xyz)

100%|██████████| 133296/133296 [01:19<00:00, 1667.87it/s]


In [21]:
df.sort_values('index', inplace=True)

Check the outputs

In [22]:
assert isclose(df.iloc[0]['u0'], -40.47893)

In [23]:
print('Parsed {} entries.'.format(len(df)))

Parsed 133296 entries.


## Remove "Evil" Molecules
These "evil" molecules are difficult to characterize because the DFT properties correspond to a different molecule than what is listed in the SMILES string, or the structure is not fully relaxed

First set are those with different SMILES strings

In [24]:
evil_mols = pd.read_csv(evil_mol_list, skiprows=9, skipfooter=1, header=None, delim_whitespace=True, engine='python',
                        names=['index', 'gdb17_smiles', 'relaxed_smiles', 'initial_smiles', 'distance'])
evil_mols = set(map(int, evil_mols['index']))
print('Found {} evil molecules'.format(len(evil_mols)))

Found 3054 evil molecules


In [25]:
original_count = len(df)
df = df[~ df['index'].apply(lambda x: x in evil_mols)]
print('Removed {} evil molecules'.format(original_count - len(df)))

Removed 3038 evil molecules


## Mark Which Entries are in the Holdout Set
Pick a random 10% of the data

In [26]:
assert len(df) == 130258

In [27]:
df['in_holdout'] = False

In [28]:
df.loc[df.sample(frac=0.1, random_state=1).index, 'in_holdout'] = True

In [29]:
print(df['in_holdout'].value_counts() / len(df))

False    0.899998
True     0.100002
Name: in_holdout, dtype: float64


*Finding*: We do have about 10% of the data in the holdout set

## Save Data to Disk
Save the data to disk pkl format

In [30]:
df.to_json(output_path, 'records', lines=True, compression='gzip')

## Create a Description of the Data
Use DLHub's mark-up capability to describe the columns

In [31]:
metadata = TabularDataset.create_model(output_path, format='json', read_kwargs={'lines': True})

Define basic metadata

In [32]:
metadata.set_title('Energetic, Band Structure, and Solubility of GDB9 Molecules Computed with G4MP2')

In [33]:
metadata.set_name('QM9-G4MP2')

In [34]:
metadata.set_authors(['Assary, Rajeev', 'Narayan, Badri', 'Cheng, Lei', 'Curtiss, Larry'], [['Argonne National Laboratory']]*4)

Set the inputs

In [35]:
metadata.annotate_column('smiles_0', 'Input smiles string', data_type='string')
metadata.annotate_column('smiles_1', 'SMILES string after relaxation', data_type='string')
metadata.annotate_column('inchi_0', 'InChi after generating coordinates with CORINA', data_type='string')
metadata.annotate_column('inchi_1', 'InChi after relaxation', data_type='string')
metadata.annotate_column('xyz', 'XYZ coordinates after relaxation', data_type='XYZ file')

In [36]:
metadata.mark_inputs(['smiles_0', 'xyz'])

Set the properties with units of Ha

In [37]:
for name, desc in [('bandgap', 'B3LYP Band gap energy'), ('homo', 'B3LYP Energy of HOMO'), ('lumo', 'B3LYP Energy of LUMO'),
                   ('zpe', 'B3LYP Zero point vibrational energy'), ('u0', 'B3LYP Internal energy at 0K'), 
                   ('u', 'B3LYP Internal energy at 298.15K'), ('h', 'B3LYP Enthalpy at 298.15K'),
                   ('u0_atom', 'B3LYP atomization energy at 0K'), ('g', 'B3LYP Free energy at 298.15K'), 
                   ('g4mp2_0k', 'G4MP2 Internal energy at 0K'), ('g4mp2_energy', 'G4MP2 Internal energy at 298.15K'),
                   ('g4mp2_enthalpy', 'G4MP2 Enthalpy at 298.15K'), ('g4mp2_free', 'G4MP2 Free eergy at 0K'), 
                   ('g4mp2_atom', 'G4MP2 atomization energy at 0K'), ('g4mp2_zpe', 'G4MP2 zero point energy')]:
    metadata.annotate_column(name, description=desc, units='Ha')

Set the remaining properties

In [38]:
metadata.annotate_column('mu', 'Dipole moment', units='D')
metadata.annotate_column('alpha', 'Isotropic polarizability', units='a_0^3')
metadata.annotate_column('R2', 'Electronic spatial extant', units='a_0^2')
metadata.annotate_column('cv', 'Heat capacity at 298.15K', units='cal/mol-K')
metadata.annotate_column('n_atom', 'Number of atoms in molecule')
metadata.annotate_column('g4mp2_hf298', 'G4MP2 Standard Enthalpy of Formation, 298K', units='kcal/mol')

The history saving thread hit an unexpected error (OperationalError('disk I/O error')).History will not be written to the database.

In [39]:
metadata.annotate_column('index', 'Index number in the database')
metadata.annotate_column('filename', 'Filename from the QM9 dataaset')
metadata.annotate_column('in_holdout', 'Whether the entry is in the pre-defined holdout set')

In [40]:
metadata.annotate_column('atomic_charges', 'Atomic charges on each atom, as predicted from B3LYP')

In [41]:
for name in ['A', 'B', 'C']:
    metadata.annotate_column(name, 'Rotational constant', units='GHz')

In [42]:
metadata.annotate_column('n_electrons', 'Numebr of electrons')
metadata.annotate_column('n_heavy_atoms', 'Number of non-hydrogen atoms')

In [43]:
assert metadata.get_unannotated_columns() == []

Set the properties

In [44]:
with open(os.path.join(os.path.dirname(output_path), 'description.json'), 'w') as fp:
    json.dump(metadata.to_dict(), fp, indent=2)